In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr # oh yeah!

load_dotenv(override=True)

openai = OpenAI()

messages = [
    {"role": "system", "content": "You are a helpful assistant that can answer questions and help with tasks."},
    {"role": "user", "content": "What is the capital of the moon?"}
]

api_key = os.getenv("OPENAI_API_KEY")
response = openai.chat.completions.create(
    model="gpt-4o-mini",

    messages=messages
)

print(response.choices[0].message.content)


The Moon does not have a capital, as it is not a sovereign nation or a territory with a governing body. The Moon is a natural satellite of Earth, and while there have been numerous missions and studies regarding human exploration and potential colonization, there is currently no established settlement or governmental structure on the Moon.


In [5]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [6]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. 
# If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

In [10]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("ed", "bananas"))

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
